In [60]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [61]:
from transformers import TextClassificationPipeline
from transformers import TFBertForSequenceClassification
from transformers import BertTokenizerFast
import tensorflow as tf

loaded_tokenizer = BertTokenizerFast.from_pretrained('Tolerblanc/klue-bert-finetuned')
loaded_model = TFBertForSequenceClassification.from_pretrained('Tolerblanc/klue-bert-finetuned', output_attentions=True)

Some layers from the model checkpoint at Tolerblanc/klue-bert-finetuned were not used when initializing TFBertForSequenceClassification: ['dropout_113']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at Tolerblanc/klue-bert-finetuned.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [62]:
text_classifier = TextClassificationPipeline(
    tokenizer=loaded_tokenizer,
    model=loaded_model,
    framework='tf',
    return_all_scores=True,
    device=0
)

## **마스킹하는 함수**

In [63]:
def hate_expression(text_classifier, tokenizer, model,text):
  output = text_classifier(text)[0]
  clean = output[0]['score']
  curse = output[1]['score']
  print(f'{text} 가 입력되었으며,')

  #모델이 혐오표현이 없다고 판단한 경우
  if clean > curse:
    print(f'모델은 이 문장을 {clean * 100}% 확률로 깨끗한 문장이라고 추론했습니다.')
    return text
  #모델이 혐오표현이 있다고 판단한 경우
  else:
    print(f'모델은 이 문장을 {curse * 100}% 확률로 욕설이나 혐오표현이 있는 문장이라고 추론했습니다.')

    #문장을 단어단위로 분류
    words = text.split()
    masked_words = []

    for word in words:
        #wordpice토큰화를 진행하고, 텍스트를 정수 배열 변환
        inputs = tokenizer.encode_plus(word, return_tensors="tf")

        #하나의 문장만 사용해서 [0]으로 표현
        outputs = model(inputs)[0]

        #정규화시키고,  일반 표현과 혐오표현에 대한 각각의 확률값을 배열로 표현
        probabilities = tf.nn.softmax(outputs, axis=-1)

        #혐오표현과 일반표현의 확률이 너무 낮은 경우 임계값 설정
        threshold = 0.07514262
        if probabilities.numpy()[0, 1] > threshold:
          label_predictions = 1
        else:
          #확률값이 높은 부분의 위치를 숫자로 표현(일반표현 0, 혐오표현 1)
          label_predictions = tf.argmax(probabilities, axis=1).numpy()[0]

        # 0: 일반, 1: 혐오
        if label_predictions ==1:
            masked_word = "*" * len(word)
            masked_words.append(masked_word)
        else:
            masked_words.append(word)

    masked_text = " ".join(masked_words)
    return masked_text

##**혐오표현 마스킹이 제대로 적용된 경우**

In [64]:
text='시바견은 너무 귀엽다'
masked_text = hate_expression(text_classifier, loaded_tokenizer , loaded_model, text)
print("마스킹:"+ masked_text)

시바견은 너무 귀엽다 가 입력되었으며,
모델은 이 문장을 99.96039271354675% 확률로 깨끗한 문장이라고 추론했습니다.
마스킹:시바견은 너무 귀엽다


In [65]:
text='새끼 고양이는 귀엽다'
masked_text = hate_expression(text_classifier, loaded_tokenizer , loaded_model, text)
print("마스킹:"+ masked_text)

새끼 고양이는 귀엽다 가 입력되었으며,
모델은 이 문장을 99.96371269226074% 확률로 깨끗한 문장이라고 추론했습니다.
마스킹:새끼 고양이는 귀엽다


In [66]:
text='아저씨 발냄새 나요'
masked_text = hate_expression(text_classifier, loaded_tokenizer , loaded_model, text)
print(masked_text)

아저씨 발냄새 나요 가 입력되었으며,
모델은 이 문장을 90.98570346832275% 확률로 깨끗한 문장이라고 추론했습니다.
아저씨 발냄새 나요


In [67]:
text='만두 몇개 시키고 올게'
masked_text = hate_expression(text_classifier, loaded_tokenizer , loaded_model, text)
print(masked_text)

만두 몇개 시키고 올게 가 입력되었으며,
모델은 이 문장을 99.66683983802795% 확률로 깨끗한 문장이라고 추론했습니다.
만두 몇개 시키고 올게


In [68]:
text='내가 세금 안내면 너 같은 동사무소에서 프린트나 하는 새끼는 월급 못받고 굶어 뒈져야 돼 병신새끼야.'
masked_text = hate_expression(text_classifier, loaded_tokenizer , loaded_model, text)
print(masked_text)

내가 세금 안내면 너 같은 동사무소에서 프린트나 하는 새끼는 월급 못받고 굶어 뒈져야 돼 병신새끼야. 가 입력되었으며,
모델은 이 문장을 99.95182752609253% 확률로 욕설이나 혐오표현이 있는 문장이라고 추론했습니다.
내가 세금 안내면 너 같은 동사무소에서 프린트나 하는 *** 월급 못받고 굶어 *** 돼 ******


In [69]:
text='이게 돈이면 다 되는 대한민국 클라쓰쥐~~~~~!!!!!'
masked_text = hate_expression(text_classifier, loaded_tokenizer , loaded_model, text)
print(masked_text)

이게 돈이면 다 되는 대한민국 클라쓰쥐~~~~~!!!!! 가 입력되었으며,
모델은 이 문장을 55.15960454940796% 확률로 욕설이나 혐오표현이 있는 문장이라고 추론했습니다.
이게 돈이면 다 되는 대한민국 클라쓰쥐~~~~~!!!!!


In [70]:
text = '저게 시대적언어면 한남충도 시대적언어 아니노 ㅋㅋ	'
masked_text = hate_expression(text_classifier, loaded_tokenizer , loaded_model, text)
print(masked_text)

저게 시대적언어면 한남충도 시대적언어 아니노 ㅋㅋ	 가 입력되었으며,
모델은 이 문장을 99.55450296401978% 확률로 욕설이나 혐오표현이 있는 문장이라고 추론했습니다.
저게 시대적언어면 **** 시대적언어 아니노 ㅋㅋ


In [71]:
text = 'ㅅㅂ'
masked_text = hate_expression(text_classifier, loaded_tokenizer , loaded_model, text)
print(masked_text)

ㅅㅂ 가 입력되었으며,
모델은 이 문장을 88.87845277786255% 확률로 욕설이나 혐오표현이 있는 문장이라고 추론했습니다.
**


In [72]:
text = '마 씨발련 아 몇평이고 맷개드갔노'
masked_text = hate_expression(text_classifier, loaded_tokenizer , loaded_model, text)
print(masked_text)

마 씨발련 아 몇평이고 맷개드갔노 가 입력되었으며,
모델은 이 문장을 99.95473027229309% 확률로 욕설이나 혐오표현이 있는 문장이라고 추론했습니다.
마 *** 아 몇평이고 맷개드갔노


In [73]:
text='무게치는거에 인생버린새끼들'
masked_text = hate_expression(text_classifier, loaded_tokenizer , loaded_model, text)
print(masked_text)

무게치는거에 인생버린새끼들 가 입력되었으며,
모델은 이 문장을 99.88462924957275% 확률로 욕설이나 혐오표현이 있는 문장이라고 추론했습니다.
무게치는거에 *******


In [74]:
text='이 ㅂㅅ은 그냥 꼼처논돈으로 잘 살아도 뭐라안하는데 눈에 안뛰였으면 버러지같은새키'
masked_text = hate_expression(text_classifier, loaded_tokenizer , loaded_model, text)
print(masked_text)

이 ㅂㅅ은 그냥 꼼처논돈으로 잘 살아도 뭐라안하는데 눈에 안뛰였으면 버러지같은새키 가 입력되었으며,
모델은 이 문장을 99.95831847190857% 확률로 욕설이나 혐오표현이 있는 문장이라고 추론했습니다.
이 *** 그냥 꼼처논돈으로 잘 살아도 뭐라안하는데 눈에 안뛰였으면 *******


In [75]:
text='나라 생긴지 몇 년 됐다고 저런 전통도 있었냐 ㅋㅋ ㅂㄹㅈ ㅋㅋㅋ'
masked_text = hate_expression(text_classifier, loaded_tokenizer , loaded_model, text)
print(masked_text)

나라 생긴지 몇 년 됐다고 저런 전통도 있었냐 ㅋㅋ ㅂㄹㅈ ㅋㅋㅋ 가 입력되었으며,
모델은 이 문장을 88.39884996414185% 확률로 욕설이나 혐오표현이 있는 문장이라고 추론했습니다.
나라 *** 몇 * 됐다고 저런 전통도 있었냐 ㅋㅋ *** ㅋㅋㅋ


In [76]:
text='니넨얼마나잘살아왔냐ㅋㅋ 열등감오지네'
masked_text = hate_expression(text_classifier, loaded_tokenizer , loaded_model, text)
print(masked_text)

니넨얼마나잘살아왔냐ㅋㅋ 열등감오지네 가 입력되었으며,
모델은 이 문장을 97.36399054527283% 확률로 욕설이나 혐오표현이 있는 문장이라고 추론했습니다.
니넨얼마나잘살아왔냐ㅋㅋ ******


In [77]:
text='이ㅂㅅ은 그냥 꼼처논돈으로 잘 살아도 뭐라안하는데 눈에 안뛰였으면 버러지같은새키'
masked_text = hate_expression(text_classifier, loaded_tokenizer , loaded_model, text)
print(masked_text)

이ㅂㅅ은 그냥 꼼처논돈으로 잘 살아도 뭐라안하는데 눈에 안뛰였으면 버러지같은새키 가 입력되었으며,
모델은 이 문장을 99.95836615562439% 확률로 욕설이나 혐오표현이 있는 문장이라고 추론했습니다.
**** 그냥 꼼처논돈으로 잘 살아도 뭐라안하는데 눈에 안뛰였으면 *******


In [78]:
text='^^ㅣ발 우리 미드 뭐야'
masked_text = hate_expression(text_classifier, loaded_tokenizer , loaded_model, text)
print("마스킹:"+ masked_text)

^^ㅣ발 우리 미드 뭐야 가 입력되었으며,
모델은 이 문장을 97.97800183296204% 확률로 욕설이나 혐오표현이 있는 문장이라고 추론했습니다.
마스킹:**** 우리 미드 뭐야


In [79]:
text='이새키보소'
masked_text = hate_expression(text_classifier, loaded_tokenizer , loaded_model, text)
print(masked_text)

이새키보소 가 입력되었으며,
모델은 이 문장을 99.84443783760071% 확률로 욕설이나 혐오표현이 있는 문장이라고 추론했습니다.
*****


In [80]:
text = '배고픈데 앞에서 만두 먹네 개시키'
masked_text = hate_expression(text_classifier, loaded_tokenizer , loaded_model, text)
print(masked_text)

배고픈데 앞에서 만두 먹네 개시키 가 입력되었으며,
모델은 이 문장을 99.68923926353455% 확률로 욕설이나 혐오표현이 있는 문장이라고 추론했습니다.
배고픈데 앞에서 만두 먹네 ***


In [81]:
text='요목조목 다 따져봐도 특출나게 잘난것도 없는 난쟁이놈이 전생에 나라라도 구했는지 ㅋㅋ 천운을 탔구나...  계속 깝치는 모습보니 안경벗기고 스크류 드라이버로 눈쑤셔버리고 싶다.'
masked_text = hate_expression(text_classifier, loaded_tokenizer , loaded_model, text)
print(masked_text)


요목조목 다 따져봐도 특출나게 잘난것도 없는 난쟁이놈이 전생에 나라라도 구했는지 ㅋㅋ 천운을 탔구나...  계속 깝치는 모습보니 안경벗기고 스크류 드라이버로 눈쑤셔버리고 싶다. 가 입력되었으며,
모델은 이 문장을 99.92628693580627% 확률로 욕설이나 혐오표현이 있는 문장이라고 추론했습니다.
요목조목 다 따져봐도 특출나게 잘난것도 없는 ***** 전생에 **** 구했는지 ㅋㅋ 천운을 탔구나... 계속 *** 모습보니 안경벗기고 스크류 드라이버로 ****** 싶다.


## **혐오표현 마스킹이 제대로 적용되지 않은 경우**

In [82]:
text = '바보'
masked_text = hate_expression(text_classifier, loaded_tokenizer , loaded_model, text)
print(masked_text)

바보 가 입력되었으며,
모델은 이 문장을 99.91893172264099% 확률로 깨끗한 문장이라고 추론했습니다.
바보


In [83]:
text='버러지들'
masked_text = hate_expression(text_classifier, loaded_tokenizer , loaded_model, text)
print(masked_text)

버러지들 가 입력되었으며,
모델은 이 문장을 79.22764420509338% 확률로 깨끗한 문장이라고 추론했습니다.
버러지들
